### Install libs

In [1]:
!pip3 install --user underthesea
!pip3 install --user nltk
!pip3 install --user bs4
!pip3 install --user pandas
!pip3 install --user matplotlib

### Define consts

In [2]:
import sqlite3 as lite
import sys
import os

import regex as re
from underthesea import word_tokenize
from bs4 import BeautifulSoup

con = None 

working_dir = "C:\\Users\\Christian\\Documents\\myProjects\\SeniorProject\\multitag\\data\\final\\"

db_name = 'development.db'
title_index = 0
tag_index = 1
html_index = 2

### Text pre-processing algorithm

In [3]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"
 
def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()

# Hàm chuyển Unicode dựng sẵn về Unicde tổ hợp (phổ biến hơn)
def convert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

bang_nguyen_am = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                  ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                  ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                  ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                  ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                  ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                  ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                  ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                  ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                  ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                  ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                  ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]
bang_ky_tu_dau = ['', 'f', 's', 'r', 'x', 'j']

nguyen_am_to_ids = {}

for i in range(len(bang_nguyen_am)):
    for j in range(len(bang_nguyen_am[i]) - 1):
        nguyen_am_to_ids[bang_nguyen_am[i][j]] = (i, j)

def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word

    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)

    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True


def chuan_hoa_dau_cau_tieng_viet(sentence):

    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)', r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)

def remove_html(txt):
    return re.sub(r'<[^>]*>', '', txt)


### Download stopwords

In [4]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Christian\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [5]:
# import regex as re
# from underthesea import word_tokenize
# from bs4 import BeautifulSoup

In [6]:
VIETNAMESE_STOPWORDS = set(stopwords.words('vietnamese_dash'))

def text_prepare(text):
    soup = BeautifulSoup(text)
    text = soup.get_text().replace('\n ',' ') 
    text = chuan_hoa_dau_cau_tieng_viet(text)
    text = convert_unicode(text)
    text = word_tokenize(text, format='text')
    text = " ".join([s for s in text.split(" ") if s not in VIETNAMESE_STOPWORDS])
    text = text.lower()
    text = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

### Read dataset files

In [7]:
from ast import literal_eval
import pandas as pd
import numpy as np

In [8]:
def read_data(filename):
    data = pd.read_csv(filename)
    data['tags'] = data['tags'].apply(literal_eval)
    return data

In [9]:
train = read_data(working_dir + 'vi_blog_train.csv')
validation = read_data( working_dir + 'vi_blog_validation.csv')
test = pd.read_csv(working_dir + 'vi_blog_test.csv')

In [10]:
# train = train[:1000] # limit train
# validation = validation[:30]
# test = test[:30]

In [11]:
train.head()

,title,tags
0,chào_mừng trở_lại blog duthaho tết_nhất dí thi...,[git]
1,hôm_nay chạy ứng_dụng docker đơn_giản ví_dụ no...,[docker]
2,thực_chất câu_chuyện anh_chàng buôn chuối viết...,[javascript]
3,đẹp trời quyết_định viết dịch_vụ web dự_định t...,[design]
4,đọc_bài viết băm mật_khẩu thaidn trường môn bả...,[security]


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36092 entries, 0 to 36091
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   36092 non-null  object
 1   tags    36092 non-null  object
dtypes: object(2)
memory usage: 564.1+ KB


In [13]:
X_train, y_train = train['title'].values, train['tags'].values
X_val, y_val = validation['title'].values, validation['tags'].values
X_test = test['title'].values

In [14]:
X_train

array(['chào_mừng trở_lại blog duthaho tết_nhất dí this cố_gắng viết tất_niên dell chữ tiếng tiếng tiêu_đề trend tiêu_đề trang_web hàng lội blog follow nổi_tiếng github star tốn lọc viết bookmark lo_lắng điện_máy xanh lo quên duthaho lo build website https bookmark duthaho com a bookmark reading list for programmers bookmark đọc hàng bot chạy cơm tự_động update viết hót share reddit phản_hồi tích_cực pr trở_lại chủ_đề viết hôm_nay  github repo star hy_vọng có_ích 1 list of free learning resources https github comebookfoundationfree programming books tiêu_đề repo chứa resources tutorial article video ebook ngôn_ngữ free _lượng resource nổi đời đọc 2 free for developers https github comripienaarfree for dev repo repo chỗ sharing danh_sách công_cụ tool mục_đích công_việc lập_trình_viên code quantity ci cd security danh_sách tool chia thể_loại thử implement project tool thú_vị free repo 3 tech interview handbook https github comyangshuntech interview handbook repo hữu_ích đổi job tết no_nê

### Test text_prepare() func

In [15]:
sen = "<p>hướng dẫn lập trình javascript, với nodejs framework</p>"
print(text_prepare(sen))

hướng_dẫn lập_trình javascript nodejs framework


### Prepare the text

In [16]:
X_train = [text_prepare(x) for x in X_train]
X_val = [text_prepare(x) for x in X_val]
X_test = [text_prepare(x) for x in X_test]

In [157]:
pickle.dump(X_train, open('x_train_prepare.ds', 'wb'))

In [17]:
X_train[:3]

['chào_mừng trở_lại blog duthaho tết_nhất dí this cố_gắng viết tất_niên dell chữ tiếng tiếng tiêu_đề trend tiêu_đề trang_web hàng lội blog follow nổi_tiếng github star tốn lọc viết bookmark lo_lắng điện_máy xanh lo quên duthaho lo build website https bookmark duthaho com a bookmark reading list for programmers bookmark đọc hàng bot chạy cơm tự_động update viết hót share reddit phản_hồi tích_cực pr trở_lại chủ_đề viết hôm_nay github repo star hy_vọng có_ích 1 list of free learning resources https github comebookfoundationfree programming books tiêu_đề repo chứa resources tutorial article video ebook ngôn_ngữ free _lượng resource nổi đời đọc 2 free for developers https github comripienaarfree for dev repo repo chỗ sharing danh_sách công_cụ tool mục_đích công_việc lập_trình_viên code quantity ci cd security danh_sách tool chia thể_loại thử implement project tool thú_vị free repo 3 tech interview handbook https github comyangshuntech interview handbook repo hữu_ích đổi job tết no_nê học đi

In [18]:
y_train

array([list(['git']), list(['docker']), list(['javascript']), ...,
       list(['c++']), list(['c++']), list(['c++'])], dtype=object)

### Tokenizer

In [19]:
from nltk.tokenize.treebank import TreebankWordTokenizer
twd = TreebankWordTokenizer()
twd.tokenize(X_train[0])

['chào_mừng',
 'trở_lại',
 'blog',
 'duthaho',
 'tết_nhất',
 'dí',
 'this',
 'cố_gắng',
 'viết',
 'tất_niên',
 'dell',
 'chữ',
 'tiếng',
 'tiếng',
 'tiêu_đề',
 'trend',
 'tiêu_đề',
 'trang_web',
 'hàng',
 'lội',
 'blog',
 'follow',
 'nổi_tiếng',
 'github',
 'star',
 'tốn',
 'lọc',
 'viết',
 'bookmark',
 'lo_lắng',
 'điện_máy',
 'xanh',
 'lo',
 'quên',
 'duthaho',
 'lo',
 'build',
 'website',
 'https',
 'bookmark',
 'duthaho',
 'com',
 'a',
 'bookmark',
 'reading',
 'list',
 'for',
 'programmers',
 'bookmark',
 'đọc',
 'hàng',
 'bot',
 'chạy',
 'cơm',
 'tự_động',
 'update',
 'viết',
 'hót',
 'share',
 'reddit',
 'phản_hồi',
 'tích_cực',
 'pr',
 'trở_lại',
 'chủ_đề',
 'viết',
 'hôm_nay',
 'github',
 'repo',
 'star',
 'hy_vọng',
 'có_ích',
 '1',
 'list',
 'of',
 'free',
 'learning',
 'resources',
 'https',
 'github',
 'comebookfoundationfree',
 'programming',
 'books',
 'tiêu_đề',
 'repo',
 'chứa',
 'resources',
 'tutorial',
 'article',
 'video',
 'ebook',
 'ngôn_ngữ',
 'free',
 '_lượng',

In [20]:
# Dictionary of all tags from train corpus with their counts.
tags_counts = {}
# Dictionary of all words from train corpus with their counts.
words_counts = {}

all_tags = []
vocabulary=[]
from collections import Counter
from nltk.tokenize.treebank import TreebankWordTokenizer
twd = TreebankWordTokenizer()

for tag in y_train:
    all_tags.extend(tag)

for txt in X_train:
    vocabulary.extend(twd.tokenize(txt))

tags_counts = Counter(all_tags)
words_counts = Counter(vocabulary)
print(tags_counts.items())

dict_items([('git', 513), ('docker', 445), ('javascript', 6937), ('design', 335), ('security', 879), ('api', 626), ('devops', 600), ('social-networking', 44), ('browser', 201), ('golang', 205), ('algorithm', 947), ('ide', 2336), ('web-server', 374), ('react-js', 1366), ('css', 1374), ('iot', 179), ('os', 186), ('regex', 148), ('agile', 281), ('ruby', 3325), ('blockchain', 355), ('php', 3534), ('cloud-computing', 686), ('oop', 967), ('testing', 2650), ('c++', 10159), ('react-native', 355), ('database', 126), ('ai', 40), ('java', 88), ('seo', 267), ('micro-services', 41), ('python', 2735), ('dot-net', 27), ('vue-js', 628), ('angular-js', 598), ('mobile', 730)])


### Transform to vector

In [21]:
import operator

TOP_WORDS=sorted_d = sorted(words_counts.items(), key=operator.itemgetter(1),reverse=True)[:5000]

In [22]:
TOP_WORDS

[('1', 151797),
 ('2', 88055),
 ('0', 79348),
 ('a', 74610),
 ('class', 64306),
 ('the', 59795),
 ('i', 59405),
 ('file', 56452),
 ('code', 52472),
 ('3', 50603),
 ('viết', 50478),
 ('function', 47789),
 ('this', 45351),
 ('return', 41457),
 ('to', 41177),
 ('ví_dụ', 41054),
 ('name', 40946),
 ('ta', 40744),
 ('dữ_liệu', 40388),
 ('ứng_dụng', 39987),
 ('int', 39379),
 ('n', 38067),
 ('user', 37670),
 ('test', 37661),
 ('end', 35936),
 ('chạy', 35834),
 ('if', 35345),
 ('hàm', 34896),
 ('in', 33793),
 ('app', 32688),
 ('thử', 31413),
 ('div', 31156),
 ('for', 30498),
 ('4', 30361),
 ('5', 28955),
 ('id', 28481),
 ('data', 27806),
 ('https', 27581),
 ('and', 27051),
 ('js', 25976),
 ('new', 25536),
 ('gọi', 25279),
 ('x', 25260),
 ('server', 25164),
 ('is', 25131),
 ('of', 24805),
 ('kiểm_tra', 24658),
 ('string', 24626),
 ('c', 23055),
 ('component', 22702),
 ('object', 22586),
 ('giúp', 22456),
 ('lỗi', 22285),
 ('public', 22115),
 ('html', 22100),
 ('http', 22038),
 ('kết_quả', 21715)

In [23]:
import operator
DICT_SIZE = 5000
ALL_WORDS =[i for i,j in sorted(words_counts.items(), key=operator.itemgetter(1),reverse=True)[:5000]]

WORDS_TO_INDEX = {}
for count, word in enumerate(ALL_WORDS):
    WORDS_TO_INDEX.update({word:count})
    
INDEX_TO_WORDS = dict((v,k) for k,v in WORDS_TO_INDEX.items())
#ALL_WORDS = WORDS_TO_INDEX.keys()

def my_bag_of_words(text, words_to_index, dict_size):

    result_vector = np.zeros(dict_size)
    
    for word in text.split():
        if word in words_to_index.keys():
            x= words_to_index[word]
            result_vector[x]=1
    return result_vector

### Test bag_of_words() func

In [24]:
def test_my_bag_of_words():
    words_to_index = {'hi': 0, 'you': 1, 'me': 2, 'are': 3}
    examples = ['hi how are you']
    answers = [[1, 1, 0, 1]]
    
    my_words = my_bag_of_words(examples[0], words_to_index, 4)
    print(my_words)
    
    for ex, ans in zip(examples, answers):
        if (my_bag_of_words(ex, words_to_index, 4) != ans).any():
            return "Wrong answer for the case: '%s'" % ex
    return 'Basic tests are passed.'

In [25]:
print(test_my_bag_of_words())

[1. 1. 0. 1.]
Basic tests are passed.


In [26]:
from scipy import sparse as sp_sparse

In [27]:
X_train_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_train])
X_val_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_val])
X_test_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in X_test])
print('X_train shape ', X_train_mybag.shape)
print('X_val shape ', X_val_mybag.shape)
print('X_test shape ', X_test_mybag.shape)

X_train shape  (36092, 5000)
X_val shape  (613, 5000)
X_test shape  (1498, 5000)


In [28]:
X_train_mybag.shape

(36092, 5000)

In [29]:
row = X_train_mybag[10].toarray()[0]
row

array([1., 1., 0., ..., 0., 0., 0.])

In [30]:
row = X_train_mybag[10].toarray()[0]
non_zero_elements_count = len(row[row != 0])

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
def tfidf_features(X_train, X_val, X_test):
   
    
    # Fit the vectorizer on the train set
    # Transform the train, test, and val sets and return the result
    
    
    tfidf_vectorizer = TfidfVectorizer(token_pattern = '(\S+)') # we want c++ or c# to be there
    X_train = tfidf_vectorizer.fit_transform(X_train)
    X_val = tfidf_vectorizer.transform(X_val)
    X_test = tfidf_vectorizer.transform(X_test)

    return X_train, X_val, X_test, tfidf_vectorizer.vocabulary_

In [33]:
X_train_tfidf, X_val_tfidf, X_test_tfidf, tfidf_vocab = tfidf_features(X_train, X_val, X_test)
tfidf_reversed_vocab = {i:word for word,i in tfidf_vocab.items()}

In [88]:
print(type(X_val))

<class 'list'>


In [34]:
tfidf_reversed_vocab

{62975: 'chào_mừng',
 263701: 'trở_lại',
 51167: 'blog',
 101062: 'duthaho',
 266907: 'tết_nhất',
 101560: 'dí',
 254600: 'this',
 88678: 'cố_gắng',
 278299: 'viết',
 266666: 'tất_niên',
 93393: 'dell',
 64029: 'chữ',
 258367: 'tiếng',
 258312: 'tiêu_đề',
 262064: 'trend',
 261469: 'trang_web',
 136115: 'hàng',
 166213: 'lội',
 115964: 'follow',
 190437: 'nổi_tiếng',
 125061: 'github',
 242401: 'star',
 267233: 'tốn',
 166108: 'lọc',
 51905: 'bookmark',
 161985: 'lo_lắng',
 292343: 'điện_máy',
 287517: 'xanh',
 161978: 'lo',
 214264: 'quên',
 53732: 'build',
 282085: 'website',
 135291: 'https',
 67726: 'com',
 28429: 'a',
 216415: 'reading',
 161004: 'list',
 116300: 'for',
 210124: 'programmers',
 293801: 'đọc',
 52275: 'bot',
 63407: 'chạy',
 88240: 'cơm',
 267695: 'tự_động',
 270125: 'update',
 136667: 'hót',
 234177: 'share',
 217326: 'reddit',
 203606: 'phản_hồi',
 266100: 'tích_cực',
 207657: 'pr',
 63928: 'chủ_đề',
 136676: 'hôm_nay',
 219858: 'repo',
 135792: 'hy_vọng',
 87894

In [35]:
from sklearn.preprocessing import MultiLabelBinarizer

In [36]:
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))

y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)

In [37]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier

In [38]:
def train_classifier(X_train, y_train):
    
    model = OneVsRestClassifier(LogisticRegression(random_state=0, max_iter=10000)).fit(X_train, y_train)
    
    return model

In [39]:
classifier_mybag = train_classifier(X_train_mybag, y_train)
classifier_tfidf = train_classifier(X_train_tfidf, y_train)

C:\Users\Christian\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Christian\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [40]:
import pickle
from joblib import dump
dump(classifier_tfidf, 'tfidf_model-flask.joblib')
dump(classifier_mybag, 'bag_model-flask.joblib')
pickle.dump(classifier_tfidf, open('tfidf_model-flask.mod', 'wb'))
pickle.dump(classifier_mybag, open('bag_model-flask.mod', 'wb'))

In [41]:
y_val_predicted_labels_mybag = classifier_mybag.predict(X_val_mybag)
y_val_predicted_scores_mybag = classifier_mybag.decision_function(X_val_mybag)

y_val_predicted_labels_tfidf = classifier_tfidf.predict(X_val_tfidf)
y_val_predicted_scores_tfidf = classifier_tfidf.decision_function(X_val_tfidf)

In [42]:
y_val_pred_inversed = mlb.inverse_transform(y_val_predicted_labels_tfidf)
y_val_inversed = mlb.inverse_transform(y_val)

for i in range(30):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        'X_val[i]',
        ','.join(y_val_inversed[i]),
        ','.join(y_val_pred_inversed[i])
    ))

Title:	X_val[i]
True labels:	ruby
Predicted labels:	ruby


Title:	X_val[i]
True labels:	javascript
Predicted labels:	javascript


Title:	X_val[i]
True labels:	javascript
Predicted labels:	


Title:	X_val[i]
True labels:	ruby
Predicted labels:	ruby


Title:	X_val[i]
True labels:	angular-js,javascript
Predicted labels:	


Title:	X_val[i]
True labels:	testing
Predicted labels:	


Title:	X_val[i]
True labels:	oop,php
Predicted labels:	oop


Title:	X_val[i]
True labels:	ruby
Predicted labels:	


Title:	X_val[i]
True labels:	angular-js
Predicted labels:	angular-js


Title:	X_val[i]
True labels:	javascript
Predicted labels:	angular-js,javascript


Title:	X_val[i]
True labels:	javascript
Predicted labels:	javascript


Title:	X_val[i]
True labels:	javascript,vue-js
Predicted labels:	vue-js


Title:	X_val[i]
True labels:	angular-js
Predicted labels:	angular-js


Title:	X_val[i]
True labels:	ruby,web-server
Predicted labels:	web-server


Title:	X_val[i]
True labels:	angular-js
Predicted labels:	j

In [133]:
classifier_mybag = pickle.load(open('bag_model-flask.mod', 'rb'))
classifier_tfidf = pickle.load(open('tfidf_model-flask.mod', 'rb'))

In [134]:
def predict_mybag(data):
    queryVal = []
    queryVal.append(data)
    queryVal = [text_prepare(x) for x in queryVal]
    vector_mybag = sp_sparse.vstack([sp_sparse.csr_matrix(my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)) for text in queryVal])
    predicted_labels_mybag = classifier_mybag.predict(vector_mybag)
    val = mlb.inverse_transform(predicted_labels_mybag)
    val = list(val[0])
    return val

In [135]:
tfidf_vectorizer2 = TfidfVectorizer(token_pattern = '(\S+)')
X = tfidf_vectorizer2.fit(X_train)
def predict_tfidf(data):
    queryVal = []
    queryVal.append(data)
    queryVal = [text_prepare(x) for x in queryVal]
    vector_return = tfidf_vectorizer2.transform(queryVal)
    predicted_labels_tfidf = classifier_tfidf.predict(vector_return)
    val = mlb.inverse_transform(predicted_labels_tfidf)
    val = list(val[0])
    return val

In [154]:
mess = "làm sao để run một image trong docker compose"

In [155]:
predict_mybag(mess)

[]

In [156]:
predict_tfidf(mess)

['docker']